In [1]:
import tensorflow

In [2]:
import cv2

In [3]:
import requests
import datetime


def postRequest(dataToSend):
    r = requests.post("http://10.10.247.124:3000/detections", data=dataToSend)
    print(r.status_code, r.reason)
    
# currTime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
# postRequest({'detectionTime':currTime, 'planeHeight':666,'planeDistance':500, 'isOurs':1, 'isShutdown': 1})
    

In [20]:
# Using webcam

import sys
import matplotlib.pyplot as plt
import datetime
import numpy as np


video_capture = cv2.VideoCapture(0)
last_observation = 0

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    
    # resized is for neural net
    resized = cv2.resize(frame, (256,256)) 
    
    
    # color for neural net
    #resized = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)
    
    #plt.imshow(resized_colored)    
    #break
    
    prediction = model.predict(np.array([flatten_arr(resized)]))[0]
    
    # Run neural network
    if(prediction[0] == 0): 
        cv2.rectangle(frame, (0, 0), (30, 30), (0, 255, 0), 2) # no Plane
        last_observation = 0
    else: 
        if(last_observation != 1): # Making sure not the same plane
            cv2.rectangle(frame, (0, 0), (30, 30), (0, 0, 255), 2) # plane !
            last_observation = 1
            # send request
            currTime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            postRequest({'detectionTime':currTime, 'planeHeight':666,'planeDistance':555, 'isOurs':1, 'isShutdown': 1})

            #time.sleep(3) # EASY EASY SOLUTION !
        

    # Display the resulting frame
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()


200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK


In [25]:
np.array([flatten_arr(resized)])[0].shape

NameError: name 'np' is not defined

In [4]:
import tensorflow
import numpy as np
import PIL
from PIL import Image
from matplotlib.pyplot import imshow



In [5]:
from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
import os

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

1.12.0-rc2


In [6]:
def flatten_arr(arr):
    num_elements = 1
    for dim in arr.shape:
        num_elements*=dim
    
    return arr.reshape(num_elements,1)

size = 256,256

In [7]:
# Rename plane
c = 1
for filename in os.listdir("./planesDataset/planePics"):
    os.rename("./planesDataset/planePics/" + filename, "./planesDataset/planePics/plane" + str(c) + ".jpg")
    c += 1


FileExistsError: [WinError 183] Cannot create a file when that file already exists: './planesDataset/planePics/plane10.jpg' -> './planesDataset/planePics/plane2.jpg'

In [8]:
# Rename not plane
c = 1
for filename in os.listdir("./planesDataset/noPlanePics"):
    os.rename("./planesDataset/noPlanePics/" + filename, "./planesDataset/noPlanePics/noPlane" + str(c) + ".jpg")
    c += 1


FileExistsError: [WinError 183] Cannot create a file when that file already exists: './planesDataset/noPlanePics/noPlane10.jpg' -> './planesDataset/noPlanePics/noPlane2.jpg'

In [9]:
# Generating data set
planes = []
noPlanes = []

size = 256,256

for filename in os.listdir("./planesDataset/planePics"):
    
    try:
    
        img = PIL.Image.open("./planesDataset/planePics/" + filename)
        img = img.resize(size, Image.ANTIALIAS)

        img = np.asarray(img)/255.0
        img = flatten_arr(img)

        planes.append([img,np.array([1,0])])


        #print(filename)
        #print("./planePics/" + filename)
    
    except:
        print("Fuck " + filename)

for filename in os.listdir("./planesDataset/noPlanePics"):
    try:
        img = PIL.Image.open("./planesDataset/noPlanePics/" + filename)
        img = img.resize(size, Image.ANTIALIAS) 

        img = np.asarray(img)/255.0
        img = flatten_arr(img)

        #print(filename)
        noPlanes.append([img,np.array([0,1])])
    except:
        print("Fuck " + filename)


In [10]:
len(noPlanes)

50

In [33]:
# Splitting data

# planes_test = planes[80:]
# noPlanes_test = planes[80:]

# planes = planes[:80]
# noPlanes = noPlanes[:80]

# len(planes),len(planes_test)


(50, 0)

In [34]:
# random selection
# new_planes = []
# new_noPlanes = []

# for p in range(0,15):
#     new_planes.append(planes[p*5])
#     new_noPlanes.append(noPlanes[p*5])
# planes = new_planes

# noPlanes = new_noPlanes

In [11]:
# Shuffling data
from random import shuffle

data = planes + noPlanes
shuffle(data)

imgdata,label = [],[]


for d in data:
    imgdata.append(d[0])
    label.append(d[1])

In [12]:
# Saving
# toucing tesnorflow's dick
imgdata = np.asarray(imgdata)
label = np.asarray(label)

np.save('imgdata.npy', imgdata)
np.save('label.npy',label)



In [14]:
# ONLY RUN THIS

imgdata = np.load('imgdata.npy')
label = np.load('label.npy')

In [15]:
print(imgdata.shape)
len(label)

(100, 196608, 1)


100

In [16]:
# Deep learningg
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(size[0]*size[1]*3,1)),
    keras.layers.Dense(400, activation=tf.nn.relu),
    keras.layers.Dense(400, activation=tf.nn.relu),
    keras.layers.Dense(400, activation=tf.nn.relu),
    keras.layers.Dense(400, activation=tf.nn.relu),
    keras.layers.Dense(400, activation=tf.nn.relu),
    keras.layers.Dense(2, activation=tf.nn.sigmoid)
])

In [17]:
model.compile(optimizer='sgd', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [18]:
model.fit(imgdata, label, epochs= 60) # 70 worked fine

Epoch 1/60
100/100 [==============================] - 4s 35ms/step - loss: 0.7217 - acc: 0.4750
Epoch 2/60
100/100 [==============================] - 2s 16ms/step - loss: 0.7923 - acc: 0.4200
Epoch 3/60
100/100 [==============================] - 2s 16ms/step - loss: 0.7437 - acc: 0.5850
Epoch 4/60
100/100 [==============================] - 2s 16ms/step - loss: 0.7308 - acc: 0.4900
Epoch 5/60
100/100 [==============================] - 2s 16ms/step - loss: 0.6037 - acc: 0.7450
Epoch 6/60
100/100 [==============================] - 2s 16ms/step - loss: 0.9269 - acc: 0.4600
Epoch 7/60
100/100 [==============================] - 2s 16ms/step - loss: 0.8630 - acc: 0.3800
Epoch 8/60
100/100 [==============================] - 2s 16ms/step - loss: 0.6385 - acc: 0.6300
Epoch 9/60
100/100 [==============================] - 2s 17ms/step - loss: 0.6581 - acc: 0.6350
Epoch 10/60
100/100 [==============================] - 2s 16ms/step - loss: 0.6916 - acc: 0.5250
Epoch 11/60
100/100 [==================

In [ ]:
# # Saving the model

# # Save the weights
# model.save('my_model.h5')

# new_model = keras.models.load_model('my_model.h5')


In [16]:
model.predict(np.array([flatten_arr(resized)]))[0]

NameError: name 'resized' is not defined

In [25]:
imgdata.shape

(199, 49152, 1)